Preparing

In [2]:
#sizes = [0.2, 0.3, 0.3, 0.3, 0.4, 0.5, 0.6]
sizes = [0.5, 0.3, 0.2, 0.4, 0.7, 0.2, 0.4, 0.8, 0.1, 0.1]


class BinPackingEnv:
    def __init__(self, sizes):
        self.sizes = sizes
        self.count = len(sizes)
        self.clearBins()

    def clearBins(self):
        self.bins = [[] for i in range(self.count)]
        self.bin_w = [0] * self.count

    def addItem(self, id : int, bin : int):
        self.bins[bin].append(id)
        self.bin_w[bin] += sizes[id]

    def isOverflow(self):
        for i in range(self.count):
            if self.bin_w[i] > 1:
                return True
        return False
    
    def countValidBin(self):
        count = 0
        for bin in self.bins:
            if len(bin) != 0:
                count += 1
        return count
        




Solvers

In [91]:
import copy
import itertools
from typing import TypeVar

class BruteForceSolver:
     def solve(self, env:BinPackingEnv):
        patterns = [[(x // pow(env.count, i) % env.count) for i in range(env.count)] for x in range(pow(env.count, env.count))]
        min_bin = env.count
        min_patterns = []

        for pattern in patterns:
            overflow = False
            env.clearBins()
            for id in range(env.count):
                env.addItem(id, pattern[id])
                if env.isOverflow():
                    overflow = True
                    break
            if not overflow:
                bin_c = env.countValidBin()
                if min_bin > bin_c:
                    min_patterns.clear()
                    min_bin = bin_c
                    min_patterns.append(pattern)
                    print("least pattern reached : ",pattern)
                elif min_bin == bin_c:
                    min_patterns.append(pattern)

        print(min_patterns)

class BruteForceSolverSingleLoop:
     def solve(self, env:BinPackingEnv):
        #patterns = [[(x // pow(env.count, i) % env.count) for i in range(env.count)] for x in range(pow(env.count, env.count))]
        min_bin = env.count
        min_patterns = []

        for pattern_id in range(pow(env.count, env.count)):
            overflow = False
            env.clearBins()
            for item_id in range(env.count):
                bin_id = pattern_id // pow(env.count, item_id) % env.count
                env.addItem(item_id, bin_id)
                if env.isOverflow():
                    overflow = True
                    break
            if not overflow:
                pattern = env.bins
                bin_c = env.countValidBin()
                if min_bin > bin_c:
                    min_patterns.clear()
                    min_bin = bin_c
                    min_patterns.append(pattern)
                    print("least pattern reached : ",pattern)
                elif min_bin == bin_c:
                    min_patterns.append(pattern)

        print(min_patterns)

Self = TypeVar("Self", bound = "ItemMask")
class ItemMask(object):
    def __init__(self, item_count : int, mask : int):
        self.mask : int = mask
        self.item_count : int = item_count


    def __or__(self, obj : Self):
        if obj.item_count != self.item_count:
            return None
        return ItemMask(self.item_count, self.mask | obj.mask)

    def __getitem__(self, key : int):
        return (self.mask >> (self.item_count - 1) - key) % 2

class BottomMask(ItemMask):
    def __init__(self, item_count : int, mask : int):
        super().__init__(item_count, mask)
        self.remain_c = item_count - self.countBottom()

    def countBottom(self):
        count = 0
        mask = self.mask
        while mask != 0:
            count += mask % 2
            mask = mask >> 1
        return count

    def isBottom(self, n):
        return self[n] == 1

    

def makeDigitMask(item_count : int, digit : int):
    return ItemMask(item_count, (1 << (item_count - 1) - digit))

class BinPatterns(object):
    def __init__(self, item_count):
        self.item_count : int = item_count
        self.bottoms : int = 0
        self.pattern : int = 0
        
    def __iter__(self):
        return self

    def __next__(self):
        bottom_mask = BottomMask(self.item_count, self.bottoms)
        bin_pattern = BinPattern(self.item_count, bottom_mask, self.pattern)
        while True:
            if bottom_mask.remain_c == 0:
                raise StopIteration()    
            cont = False        
            for i in range(self.item_count):
                bin_count = 0
                if bottom_mask.isBottom(i):
                    for j in range(i + 1, self.item_count):
                        if bin_pattern[bin_count][j] == 1:
                            cont = True
                            break
                    bin_count += 1
            if not cont:
                print(self.pattern, self.bottoms)
                self.pattern += 1
                if self.pattern >= pow(bottom_mask.remain_c, bottom_mask.remain_c):
                    self.bottoms += 1
                    self.pattern = 0
                return bin_pattern
            self.pattern += 1
            if self.pattern >= pow(bottom_mask.remain_c, bottom_mask.remain_c):
                self.bottoms += 1
                self.pattern = 0

            bin_pattern = BinPattern(self.item_count, bottom_mask, self.pattern)


        
            

        
class BinPattern(object):
    def __init__(self, item_count : int, bottom_mask : BottomMask, pattern : int):
        self.item_count : int = item_count
        self.bins : int = 0

        remain_c : int = bottom_mask.remain_c
        remains : list[int] = []
        for i in range (item_count):
            if not bottom_mask.isBottom(i):
                remains.append(i)

        bottom_count : int = 0
        for digit in range(item_count):
            if bottom_mask.isBottom(digit):
                chunk_shift = bottom_count * item_count
                digit_shift = (item_count - 1) - digit
                self.bins |= 1 << chunk_shift << digit_shift
                for i in range(remain_c):
                    if pattern // pow(remain_c ,i) % remain_c == bottom_count:
                        digit_shift = (item_count - 1) - remains[i]
                        self.bins |= 1 << chunk_shift << digit_shift
                bottom_count += 1
        
        

    def __getitem__(self, key):
        print('bins:', self.bins, ' item_count:', self.item_count, ' key:', key)
        chunk_size = ()
        mask = (self.bins >> (key * self.item_count)) % (1 << self.item_count)
        return ItemMask(self.item_count, mask)



class BinPatternsComb:
    def __init__(self, item_count):
        None

def getAllComb(dataset : set[int]):
    data_count = len(dataset)
    comb : list[list[set[int]]] = []
    if len(dataset) == 1:
        tmp = [[dataset]]
        return tmp
    for i in range(1, data_count):
        for pattern in itertools.combinations(dataset, i):
            for remain_comb in getAllComb(dataset.difference(pattern)):
                comb.append(remain_comb)
    
    comb.append([dataset])
    return comb

tset = {i for i in range(3)}
k = 0
for pattern in getAllComb(tset):
    k += 1
    print(pattern)
print(k)

        

#class OptimizedBruteForceSolver:
#    def solve(self, env:BinPackingEnv):
#        for i in range(env.count + 1):
#            for pattern in itertools.combinations([i for i in range(env.count)], ):

    

[[{2}]]
[[{1}]]
[[{2}]]
[[{0}]]
[[{1}]]
[[{0}]]
[[{2}]]
[[{1}]]
[[{0}]]
[{2}]
[{1}]
[{1, 2}]
[{2}]
[{0}]
[{0, 2}]
[{1}]
[{0}]
[{0, 1}]
[{2}]
[{1}]
[{0}]
[{0, 1, 2}]
13


In [16]:
class FirstFitSolver:
    def solve(self, env : BinPackingEnv):
        for item_id in range(env.count):
            item_size = env.sizes[item_id]
            for bin_id in range(env.count):
                bin_space = 1 - env.bin_w[bin_id]
                if bin_space >= item_size:
                    env.addItem(item_id, bin_id)
                    break
        print(env.bins)

In [17]:
class BestFitSolver:
    def solve(self, env : BinPackingEnv):
        for item_id in range(env.count):
            item_size = env.sizes[item_id]
            best_bin_id = -1
            best_bin_space = 1.0
            for bin_id in range(env.count):
                bin_space = 1 - env.bin_w[bin_id] - item_size
                if bin_space >= 0 and bin_space < best_bin_space:
                    best_bin_id = bin_id
                    best_bin_space = bin_space
            print(item_id, best_bin_id)
            env.addItem(item_id, best_bin_id)
        print(env.bins)

In [18]:
def greatestInRemain(sizes, space):
    greatest_size = 0
    greatest_id = -1
    for item_id in range(len(sizes)):
        item_size = sizes[item_id]
        if item_size <= space and greatest_size < item_size:
            greatest_id = item_id
            greatest_size = item_size
    return greatest_id



class KyanioSolver:
    def solve(self, env : BinPackingEnv):
        items = env.sizes.copy()
        item_ids = [i for i in range(len(items))]
        bin_id = 0
        while len(items) > 0:
            space = 1 - env.bin_w[bin_id]
            greatest_id = greatestInRemain(items, space)
            items.pop(greatest_id)
            item_id = item_ids.pop(greatest_id)
            env.addItem(item_id, bin_id)
            while True:
                space = 1 - env.bin_w[bin_id]
                greatest_id = greatestInRemain(items, space)
                if greatest_id == -1:
                    break
                items.pop(greatest_id)
                item_id = item_ids.pop(greatest_id)
                env.addItem(item_id, bin_id)
            bin_id += 1
        
        print(env.bins)




Test

In [19]:
%%time

#問題を解くためのシミュレート環境を作成
bpenv = BinPackingEnv(sizes)


c = bpenv.countValidBin()
#アルゴリズム1総当たり
solver1 = BruteForceSolverSingleLoop()
#実行
solver1.solve(bpenv)

#solver2 = FirstFitSolver()
#solver2.solve(bpenv)

#solver3 = BestFitSolver()
#solver3.solve(bpenv)

#solver4 = KyanioSolver()
#solver4.solve(bpenv)

: 

: 